# Libraries and classes

In [1]:
# Libraries
import numpy as np
import laspy as lp
import scipy
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy.spatial import KDTree, ckdtree
from matplotlib.cm import tab20 as cmap

In [2]:
class Point:

    index = None
    position = None
    paths = []
    network = None
    vec = None
    linked_to = None
    treeID = None

    def __init__(self, index, position):
        self.index = index
        self.position = position

    def add_path(self, Path):
        self.paths = np.append(self.paths, Path)

In [3]:
class Path:

    index = None
    points = []
    network = None

    def __init__(self, index):
        self.index = index

    def add_point(self, this_point):
        self.points = np.append(self.points, this_point)

    def all_points_position(self):
        points_pos = np.c_[[],[],[]]
        for Point in self.points:
            points_pos = np.append(points_pos, np.c_[Point.position], axis=0)
        return points_pos

In [4]:
class Network:

    index = None
    paths = []
    points = []
    top = None

    def __init__(self, index):
        self.index = index

    def add_path(self, Path):
        self.paths = np.append(self.paths, Path)
        Path.Network = self
        for Point in Path.points:
            Point.Network = self
        self.points = np.append(self.points, Path.points)

    def size(self):
        points = np.array(())
        for Path in self.paths:
            for Point in Path.points:
                points = np.append(points, Point)
        return len(points)

    def all_paths(self):
        all_paths = np.array(())
        for Path in self.paths:
            all_paths = np.append(all_paths, Path.index)
        return all_paths

    def all_points_position(self):
        points_pos = np.array(())
        for Point in self.points:
            points_pos = np.append(points_pos, point.position)
        points_pos = np.reshape(points_pos, (-1, 3))
        return points_pos

# Insert the parameters for the code

In [5]:
# Insert the desired Las file here.
las_file = lp.file.File('trees_122_485.las', mode="r")
# name  your output
output_las = "ALL_TILES_FINAL_OUTPUT_final.las"
# Parameters (all values are in meters)
r = 4  # radius of the search spehere for the initial clustering
radius = 0.8  # the radius on which we count the point density in x and y for each point (the parameter used for local maxima calculation)
window_size = 6  # the size of the search window for local maxima in each cluster
max_distance = 0.8  # the delineated trunks radius
restrict_d = 6  # the minimum eucledian distance that 2 peaks of the same cluster can have
small_clusters = 100  # the size of the small custers we suspect as outliers (won't be deleted, they will just merge with a nearby big cluster if there is any, else they will be taken as individual clusters)
small_outliers = 30  # OPTIONAL! The minimal cluster size to be allowed as a tree. Deleting every cluster below this value.


In [6]:
# CHECK THE END OF THE CODE TO CHOOSE THE TYPE OF THE OUTPUT !

# 1. Load and Prepare the data

In [7]:
points = []

# concatenate the file coordinates
coord = np.c_[las_file.x, las_file.y, las_file.z]

# turnk the coordinates into full integers
x_new = (las_file.x).astype('int')
y_new = (las_file.y).astype('int')
z_new = (las_file.z).astype('int')

# Reduce the data, get a flat voxel index. Lowers the number of points feed to the algorithm.
new_coords = x_new + y_new * x_new.max() + z_new * y_new.max() * x_new.max()
_, sl = np.unique(new_coords, return_index=True)
coord = coord[sl,:]

# sort the coordinates by z value
coord_sorted = coord[coord[:, 2].argsort()]
position = coord_sorted

# points is a list of "point" class for each set of coordinates
# i= index/position and coord_sorted as a position
for i in range(len(coord_sorted)):
    i = Point(i, coord_sorted[i])
    points.append(i)
# create an ordered array with the size of remaning coord
index = np.arange(len(coord))

# 2. Find centroids of point clusters and tree peaks

1. A collection of points in 3D space is given, with a manually input radius value.
2. The code finds groups of points that are within the radius of each other, and it computes their group centroids.
3. For each group, it finds the point with the highest Z-value (i.e., the top of the tree), and links it to the centroid.
4. The code outputs the index of the closest point to the centroid for each group, and whether each point is the highest point of its group (i.e., at the top of the tree).

Reasons why the code runs fast
1. Using list comprehension to find the neighbors with a higher z value rather than creating a new numpy array and using numpy's boolean indexing to find these neighbors.
2. Checking for cases where a point has no neighbors within radius r first to avoid unnecessary calculations.
3. Initializing the links and centroids arrays to the same length as the position array with zeros to avoid needing to use np.append() inside the loop.

In [8]:
links = np.zeros(len(position), dtype=int)
centroids = np.zeros((len(position), 3))
has_parent = np.zeros(len(position), dtype=bool)

# Find all points within distance r of point(s) x
tree = scipy.spatial.cKDTree(position)
nn = tree.query_ball_point(position, r)

# Loop over all points
for i, this_nn in enumerate(nn):
    # If the point has no neighbors within radius r, it is a tree peak
    if len(this_nn) == 1:
        links[i] = i
        centroids[i] = position[i]
        has_parent[i] = True
    # If the point has at least one neighbor within radius rlink_nn
    else:
        # Find all neighbors with a higher z value
        upper_nnbs = [j for j in this_nn if position[j, 2] > position[i, 2]]
        # If there are no such neighbors, the point is a tree peak
        if not upper_nnbs:
            links[i] = i
            centroids[i] = position[i]
            has_parent[i] = True
        # If there are any neighbors with a higher z value
        else:
            # Calculate the centroid of the group of neighbors
            c = np.mean(position[upper_nnbs], axis=0)
            centroids[i] = c
            # Calculate the distances between each neighbor and the centroid
            dist = scipy.spatial.distance.cdist(position[upper_nnbs], [c],
                                                metric="euclidean")
            # Find the neighbor closest to the centroid and store its index as a link
            links[i] = upper_nnbs[np.argmin(dist)]

# Convert links to integer type
link_nn = links.astype(int)
has_parent = has_parent.astype('int')

# 3. Label the points

1. For each point, the code checks if it has already been assigned to a path.
2. If not, it creates a new path and adds the current point to it.
3. It then follows the links created in Part 1 to add more points to the path, until it reaches a point with no parent (i.e., at the top of the tree), at which point it ends the path.
4. If the code encounters a point that is already in a path, it creates a new network that includes both the new path and the existing path.

In [9]:
networks = []
all_paths = []
for p in points:
    current_idx = p.index

    if len(points[current_idx].paths) == 0:
        end = False

        # initialize new path
        new_path = Path(len(all_paths))  # len paths as index
        all_paths.append(new_path)

        # add first point to the path
        new_path.add_point(points[current_idx])
        points[current_idx].add_path(new_path)

        # append path
        while end == False:

            # point has a parent
            if has_parent[current_idx] != 1:

                # make link
                points[current_idx].linked_to = points[link_nn[current_idx]]

                if len(points[current_idx].linked_to.paths) == 0:

                    # not in path
                    points[current_idx].linked_to.add_path(new_path)
                    new_path.add_point(points[current_idx].linked_to)
                    current_idx = link_nn[current_idx]

                else:
                    # in path
                    points[current_idx].linked_to.Network.add_path(new_path)
                    points[current_idx].add_path(new_path)
                    points[current_idx].linked_to.add_path(new_path)
                    end = True

            # point has no parent
            # make network, end path
            else:
                points[current_idx].linked_to = points[current_idx]
                # init new network
                new_network = Network(len(networks))  # len networks as index
                new_network.add_path(new_path)  # path and points are assigned to the network
                new_network.top = current_idx
                new_network.points = new_path.points  # add points to the network
                networks.append(new_network)
                points[current_idx].Network = new_network
                end = True

# 4. Get the labels array

In [10]:
# new np array to extract and store all our individual tree labels from
labels = np.zeros(len(points))
centroids = np.array(())
size = np.array(())
# extract the label value from class network to our new built array
for p in points:
    labels[p.index] = p.Network.index
labels = labels.astype('int')

# Remove all the outlier clusters

In [11]:
array_test = np.column_stack((position, labels))

In [12]:
# Get the count of each cluster label
labels_new = array_test[:, 3]
array = array_test[:, 0:3]
unique, counts = np.unique(labels_new, return_counts=True)

# Create a dictionary to store the count of each label
label_count = dict(zip(unique, counts))

# Initialize an empty list to store the indices of the large clusters
large_cluster_indices = []

# Iterate through the cluster labels
for i, label in enumerate(labels_new):
    # If the label corresponds to a large cluster, add the index to the list
    if label_count.get(label, 0) >= 10:
        large_cluster_indices.append(i)

# Use the indices of the large clusters to create a new array
array_test = array[large_cluster_indices, :]

# Add the labels as the last column of the new array
array_test = np.column_stack((array_test, labels_new[large_cluster_indices]))

# Fix the small clusters

In [13]:
# Prepare the array for the "fix small clusters" code
labels_2 = array_test[:, 3].astype('int')
labels33, point_count33 = np.unique(labels_2, return_counts=True)

In [14]:
iterating_array = []
for i in range(len(labels33)):
    if point_count33[i] <= small_clusters:
        iterating_array.append(labels33[i])

In [15]:
# Get centroids of all clusters in the dataset
all_centroids = []
all_labs=[]
for label in np.unique(array_test[:, 3]):
    centroid = array_test[array_test[:, 3] == label, :2].mean(axis=0)
    all_centroids.append(centroid)
    all_labs.append(label)

In [16]:
# find the pairs of the closest clusters

tree1 = KDTree(all_centroids)

labels_nn = []
for i in range(len(all_labs)):
    point_cent = all_centroids[i]
    dist, idx = tree1.query(point_cent, k=2)
    closest_idx = idx[1] if idx[0] == i else idx[0]
    labels_nn.append([all_labs[i], all_labs[closest_idx ]])

# filter the list so it contains only the small clusters that we will fix
filtered_list = [x for x in labels_nn if int(x[0]) in iterating_array]
array_test2 = array_test.copy()

In [17]:
# Some other parameters for tree merging
diff_height = 1.5  # the difference in height between 2 clusters very close to each other (this is the parameter to take care of branches that are classified as a separate cluster)
branch_dist = 0.8  # the max distance a branch clsuter can be from the main tree
min_dist_tree = 1  # the max distance of 2 clusters to be checked if they are the same tree

for i in filtered_list:
    coord_xy = array_test2[array_test2[:, 3] == i[0]]
    coord_xy2 = array_test2[array_test2[:, 3] == i[1]]
    wk = distance.cdist(coord_xy[:, :2], coord_xy2[:, :2], 'euclidean')
    z = abs(coord_xy[:, 2:3].min()-coord_xy[:, 2:3].min())
    kk = array_test2[:, 2][array_test2[:, 3] == i[1]]
    z = abs(coord_xy[:, 2:3].min() - kk.min())
    if len(array_test2[array_test2 == i[0]]) < (small_clusters/2) and wk.min() < min_dist_tree :
        array_test[:, 3][array_test[:, 3] == i[0]] = i[1]
    if wk.min() < branch_dist and z > diff_height:
        array_test[:, 3][array_test[:, 3] == i[0]] = i[1]
    if len(array_test2[array_test2 == i[0]]) < small_clusters and wk.min() < min_dist_tree/2:
        array_test[:, 3][array_test[:, 3] == i[0]] = i[1]
    coord_xy = []
    coord_xy2 = []
    wk = []
    ind = []

# Optional! Delete small clusters

In [18]:
# Get the count of each cluster label
labels_new = array_test[:, 3]
array = array_test[:, 0:3]
unique, counts = np.unique(labels_new, return_counts=True)

# Create a dictionary to store the count of each label
label_count = dict(zip(unique, counts))

# Initialize an empty list to store the indices of the large clusters
large_cluster_indices = []

# Iterate through the cluster labels
for i, label in enumerate(labels_new):
    # If the label corresponds to a large cluster, add the index to the list
    if label_count.get(label, 0) >= small_outliers:
        large_cluster_indices.append(i)

# Use the indices of the large clusters to create a new array
array_test = array[large_cluster_indices, :]

# Add the labels as the last column of the new array
array_test = np.column_stack((array_test, labels_new[large_cluster_indices]))

# 6. Get the number of points in buffer per point (the local maxima column)

In [19]:
# Input data
points = array_test[:, :2]

# Create KDTree from points
kd_tree = KDTree(points)

# Array to store the number of points in the buffer for each point
count = np.zeros(len(points), dtype=int)

# Loop over each point and find points in the buffer
for i, p in enumerate(points):
    idx = kd_tree.query_ball_point(p, radius)
    count[i] = len(idx) - 1

# 7. Find the tree trunks

In [20]:
# create the full array
full_array = np.column_stack((array_test, count))


def cluster_local_maxima1(full_array, window_size, max_distance, restrict_d):
    # get the unique label of tree clusters
    unique_clusters = np.unique(full_array[:, 3])
    current_label = 1
    labels = np.zeros(full_array.shape[0], dtype=np.int64)
    full_array = np.column_stack((full_array, labels))
    iteration=0
    # Iterate through every single tree cluster separately
    for cluster_id in unique_clusters:
        peaks1=[]
        dist_peaks = 100
        # Form an array for the cluster of this iteration
        kot_arr = full_array[full_array[:, 3] == cluster_id]
        x1 = kot_arr[:, 0]
        y1 = kot_arr[:, 1]
        z1 = kot_arr[:, 2]
        p1 = kot_arr[:, 4]
        labels_k = kot_arr[:, 5]
        # Now we iterate through each point of the cluster of this iteration
        for i in range(len(kot_arr)):
            # We form a seach window around each point of the cluster
            x_min = x1[i] - window_size
            x_max = x1[i] + window_size
            y_min = y1[i] - window_size
            y_max = y1[i] + window_size
            in_window = np.bitwise_and(x1 >= x_min, x1 <= x_max)
            in_window = np.bitwise_and(in_window, np.bitwise_and(y1 >= y_min, y1 <= y_max))
            in_window = np.bitwise_and(in_window, kot_arr[:, 3] == cluster_id)

            # Calculate and save the distances between the local maximas we find.
            if len(peaks1) > 0:
                this_point = [x1[i],y1[i]]
                peak_array = np.array(peaks1)
                this_point = np.array(this_point)
                this_point = this_point.reshape(1, 2)
                dist_peaks = distance.cdist(peak_array, this_point, 'euclidean')
                
            # We find the local maximas for each window
            # Then we restric every local maximas that are way too close with each other with the parameter "restrict_d"
            # Then the local maximas with an accepted distace between each other are relabeld as a unique number for each unique tree.
            if np.max(p1[in_window]) == p1[i] and np.min(dist_peaks) > restrict_d:
                peaks1.append([x1[i], y1[i]])
                points_to_label = np.argwhere(np.logical_and(np.abs(x1-x1[i]) <= max_distance, np.abs(y1-y1[i]) <= max_distance))
                points_to_label = points_to_label.flatten()
                if labels_k[i] == 0:
                    labels_k[points_to_label] = current_label
                    current_label += 1
                else:
                    labels_k[points_to_label] = labels_k[i]

        # we create a new array with the new labels for trunks
        new_2 = np.c_[x1,y1,z1,labels_k]
        if iteration == 0:
            final_result = new_2
        else:
            final_result = np.vstack((final_result, new_2))
        iteration=1

    return final_result


Final_labels = cluster_local_maxima1(full_array, window_size, max_distance, restrict_d)

In [21]:
# Get the number of trees in this las file
tree_count = np.unique(Final_labels[:, 3])
print("there are", len(tree_count), "trees in this area")

there are 2392 trees in this area


# Save the trunk Point Cloud as a new Las file

In [22]:
lb = Final_labels
c = Final_labels[:, 3]

vals = np.linspace(0, 1, 100)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20(vals))
header = lp.header.Header()
header.data_format_id = 2
new_las = lp.file.File(output_las, mode='w', header=header)
new_las.header.scale = [0.01, 0.01, 0.01]
new_las.header.offset = [lb[:, 0].min(), lb[:, 1].min(), lb[:, 2].min()]
new_las.x = lb[:, 0]
new_las.y = lb[:, 1]
new_las.z = lb[:, 2]
new_las.pt_src_id = c.astype('uint16')
new_las.close()

# ALTERNATIVE OUTPUT
# Getting only one point(centroid) in X and Y for each tree trunk

In [ ]:
# as a dictionary

In [38]:
'''
Centroid_tree = np.unique(Final_labels[:, 3])[1:]
centroids_coord = {}

# Iterate through each cluster and find the centroid
for label in Centroid_tree:
    cluster_points = Final_labels[Final_labels[:, 3] == label][:, :2]
    centroid = np.mean(cluster_points, axis=0)
    centroids_coord[label] = centroid

#print(centroids_coord)
'''

In [ ]:
# as a numpy array

In [23]:
# Get the unique cluster labels excluding label zero
Centroid_tree = np.unique(Final_labels[:, 3])[1:]
# Initialize an empty list to store the centroids for each cluster
centroids_array  = []

# Iterate through each cluster and find the centroid
for label in Centroid_tree:
    cluster_points = Final_labels[Final_labels[:, 3] == label][:, :2]
    centroid = list(np.mean(cluster_points, axis=0))
    centroids_array.append([centroid[0], centroid[1], label])

centroids_array = np.array(centroids_array)

# Save only the tree centroids as 2D points (LAS FILE)

In [18]:
output_name = "Output_centroids_only.las"
z_value = np.zeros(centroids_array.shape[0], dtype=np.int64)
lab = centroids_array
c = centroids_array[:, 2]

vals = np.linspace(0, 1, 100)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20(vals))
header = lp.header.Header()
header.data_format_id = 2
new_las = lp.file.File(output_name, mode='w', header=header)
new_las.header.scale = [0.01, 0.01,0.01]
new_las.header.offset = [lab[:, 0].min(), lab[:, 1].min(),z_value.min()]
new_las.x = lab[:, 0]
new_las.y = lab[:, 1]
new_las.z = z_value
new_las.pt_src_id = c.astype('uint16')
new_las.close()

# Save only the tree centroids as 2D points (SHAPEFILE)

In [24]:
import shapefile
# Create a new shapefile
sf = shapefile.Writer("all_tree_centroids_out", shapeType=shapefile.POINT)

# Define the fields for the shapefile
sf.field("label", "N")

# Iterate through each row of the array and add a point to the shapefile
for row in centroids_array:
    # Extract the x, y, and label values from the row
    x, y, label = row

    # Add a point to the shapefile with the x and y coordinates
    sf.point(x, y)

    # Set the attributes for the point
    sf.record(label)

# Save and close the shapefile
sf.close()